# Molecular-Geometry-Analysis
### Code Author: Pratiksha Balasaheb Gaikwad

In [1]:
import numpy as np
import re

In [2]:
output = open('1_output.txt', 'w')

### Step 1: Read the Coordinate Data from Input

In [3]:
f = open('acetaldehyde_input.txt', 'r')

# read first line == total number of electrons in molecule
natoms = int(f.readline()) 
geom = np.zeros((natoms,3))
# unpack the atomic numbers and x, y, z coordinates
# from the above file object 'f'
atom_num, geom[:,0], geom[:,1],geom[:,2] = np.loadtxt(f, unpack = True)
atom_num = atom_num.astype(int)

# Write to the output file
output.write(f"Number of atoms: {natoms}")
output.write("\nInput cartesian coordinates: \n")

coords = np.array(list(zip(atom_num, geom[:,0],geom[:,1],geom[:,2])))
output.write(re.sub('( \[|\[|\])', '', str(coords)))


335

### Step 2: Bond Lengths

In [4]:
output.write("\n\nInteratomic distances: \n")
output.write("\natom1 \t atom2 \t Bond length (Bohr)\n")
R = np.zeros([natoms,natoms])

for i in np.arange(0, natoms):
    for j in np.arange(0, i, 1):
        x = (geom[j][0] -geom[i][0]);
        y = (geom[j][1] -geom[i][1]);
        z = (geom[j][2] -geom[i][2]);

        R[i][j] = np.sqrt(x**2 + y**2 + z**2 )
        output.write(f"\n{i} \t {j} \t {R[i][j]}")
 
# Distance matrix should be symmetric
for i in np.arange(0, natoms):
    for j in np.arange(0, i, 1):
        R[j][i] = R[i][j]


### Step 3: Bond Angles

In [5]:
def uvector(dim, atom1, atom2):
    if R[atom1][atom2] == 0:
        return 0
    else:
        return -(geom[atom1][dim] - geom[atom2][dim])/R[atom1][atom2]

In [6]:
def bond_angle(i,j,k):
    return np.arccos(uvector(0,j,i)*uvector(0,j,k) + uvector(1,j,i)*uvector(1,j,k) + uvector(2,j,i)*uvector(2,j,k))

In [7]:
output.write("\n\nBond Angles \n")
for i in np.arange(0, natoms):
    for j in np.arange(0, i):
        for k in np.arange(0,j):
            if (R[k][j] < 4.0 and R[j][i] < 4.0):
                
                output.write(f"{i}-{j}-{k} \t {bond_angle(i,j,k)*180.0/np.arccos(-1.0)}\n")
        

### Step 4: Out-of-Plane Angles

In [8]:
#A_outofplane = np.array()

# Calculate cross product
output.write("\n\nOut-of-Plane Angles \n")
for i in np.arange(0, natoms):
    for k in np.arange(0, natoms):
        for j in np.arange(0, natoms):
            for l in np.arange(0, j):
                ejkl_x = uvector(1, k,j)*uvector(2,k,l) - uvector(2,k,j)*uvector(1,k,l)
                ejkl_y = uvector(2, k,j)*uvector(0,k,l) - uvector(0,k,j)*uvector(2,k,l)
                ejkl_z = uvector(0, k,j)*uvector(1,k,l) - uvector(1,k,j)*uvector(0,k,l)
 
                exx = ejkl_x * uvector(0,k,i)
                eyy = ejkl_y * uvector(1,k,i)
                ezz = ejkl_z * uvector(2,k,i)
                
                theta = (exx + eyy + ezz)/np.sin(bond_angle(j,k,l))
                
                if theta < -1.0:
                    A_outofplane = np.arcsin(-1.0)
                elif theta > 1.0:
                    A_outofplane = np.arcsin(1.0)
                else: 
                    A_outofplane = np.arcsin(theta)
                
                if (i!=j and i!=k and i!=l and j!=k  and j!=l and k!=l and 
                    R[i][k] < 4.0 and R[k][j] < 4.0 and R[k][l] < 4.0):

                    output.write(f"{i}-{j}-{k}-{l} \t {A_outofplane*180.0/np.arccos(-1.0)}\n")
                    

### Step 5: Torsion/Dihedral Angles

In [9]:

output.write("\n\nTorsion/Dihedral Angles \n")
for i in np.arange(0, natoms):
    for j in np.arange(0, i):
        for k in np.arange(0, j):
            for l in np.arange(0, k):
                eijk_x = uvector(1, j,i)*uvector(2,j,k) - uvector(2,j,i)*uvector(1,j,k)
                eijk_y = uvector(2, j,i)*uvector(0,j,k) - uvector(0,j,i)*uvector(2,j,k)
                eijk_z = uvector(0, j,i)*uvector(1,j,k) - uvector(1,j,i)*uvector(0,j,k)
                
                ejkl_x = uvector(1, k,j)*uvector(2,k,l) - uvector(2,k,j)*uvector(1,k,l)
                ejkl_y = uvector(2, k,j)*uvector(0,k,l) - uvector(0,k,j)*uvector(2,k,l)
                ejkl_z = uvector(0, k,j)*uvector(1,k,l) - uvector(1,k,j)*uvector(0,k,l)
                
                exx = eijk_x * ejkl_x
                eyy = eijk_y * ejkl_y
                ezz = eijk_z * ejkl_z
                
                #if np.sin(bond_angle(i,j,k))!= 0 and np.sin(bond_angle(j,k,l)) != 0:
                tau = (exx + eyy + ezz)/(np.sin(bond_angle(i,j,k))*np.sin(bond_angle(j,k,l)))
                if tau < (-1.0):
                    tau = np.arccos(-1.0)
                elif tau > 1.0:
                    tau = np.arccos(1.0)
                else: 
                    tau = np.arccos(tau)
                    
                rsltx = eijk_y * ejkl_z - eijk_z * ejkl_y
                rslty = eijk_z * ejkl_x - eijk_x * ejkl_z
                rsltz = eijk_x * ejkl_y - eijk_y * ejkl_x
                
                norm = rsltx**2 + rslty**2 + rsltz**2
                if norm != 0:
                    rsltx /= norm
                    rslty /= norm
                    rsltz /= norm

                dot_prdct = rsltx*uvector(0,j,k) + rslty*uvector(1,j,k) + rsltz*uvector(2,j,k)
                
                if dot_prdct < 0.0:
                    tau = tau*-1.0
                
                if (R[i][j] < 4.0 and R[j][k] < 4.0 and R[k][l] < 4.0):
                    output.write(f"{i}-{j}-{k}-{l} \t {tau*180.0/np.arccos(-1.0)}\n")

### Step 6: Center-of-Mass  Translation

In [10]:
def assign_mass(atom_num):
    if atom_num == 1:
        return 1.00782503223
    elif atom_num == 6:
        return 12.0
    elif atom_num == 8:
        return 15.99491461957

In [11]:
def translate(Xcm, Ycm, Zcm):
    geom[:,0] -= Xcm
    geom[:,1] -= Ycm
    geom[:,2] -= Zcm

In [12]:
mass = np.zeros(natoms)
for i in range(natoms):
    mass[i] = assign_mass(atom_num[i])

sumx = 0.0
sumy = 0.0
sumz = 0.0
for i in range(natoms):
    sumx += mass[i]*geom[i][0]    
    sumy += mass[i]*geom[i][1]    
    sumz += mass[i]*geom[i][2]
    
Xcm = sumx/sum(mass)
Ycm = sumy/sum(mass)
Zcm = sumz/sum(mass)


# Translate the coordinates according to COM
translate(Xcm, Ycm, Zcm)
output.write("\n\nMolecular center of mass:\n")
output.write(f"Xcm = {Xcm} \t Ycm = {Ycm} \t Zcm = {Zcm}\n")

64

### Step 7: Principal Moments of Inertia

In [13]:
MoI = np.zeros((3,3))
output.write("\n\nMoment of Inertia (amu bohr^2):\n")

MoI[0][0] += sum(mass*(geom[:,1]**2+geom[:,2]**2))
MoI[1][1] += sum(mass*(geom[:,0]**2+geom[:,2]**2))
MoI[2][2] += sum(mass*(geom[:,0]**2+geom[:,1]**2))
MoI[0][1] = - sum(mass*geom[:,0]*geom[:,1])
MoI[1][2] = -sum(mass*geom[:,1]*geom[:,2])
MoI[2][0] = -sum(mass*geom[:,2]*geom[:,0])

MoI[1][0] = MoI[0][1]
MoI[2][1] = MoI[1][2]
MoI[0][2] = MoI[2][0]

for i in range(3):
    output.write(f"{MoI[i][0]} \t {MoI[i][1]} \t {MoI[i][2]}\n")

    
eigvals, eigvecs = np.linalg.eig(MoI)
eigvals = np.sort(eigvals)
output.write("\n\nPrincipal Moment of Intertia (amu bohr^2)\n")
output.write(f"{eigvals[0]} \t {eigvals[1]} \t {eigvals[2]}\n")

conv_factr = 0.529177210903 * 0.529177210903 #amu to angstrom
output.write("\n\nPrincipal Moment of Intertia (amu AA^2)\n")
evals = eigvals*conv_factr
output.write(f"{evals[0]} \t {evals[1]} \t {evals[2]}\n")


conv = 1.6605402E-24 * 0.529177249E-8 * 0.529177249E-8;
output.write("\n\nPrincipal Moment of Intertia (g cm^2)\n")
evals = eigvals*conv
output.write(f"{evals[0]} \t {evals[1]} \t {evals[2]}\n")


70

In [14]:
# Classification of Rotor
if (natoms ==2):
    output.write("\nMolecule is diatomic.\n")
elif (eigvals[0]< 1e-4):
    output.write("\nMolecule is linear.\n")
elif (abs(eigvals[0]-eigvals[1]) < 1e-4 and abs(eigvals[1]-eigvals[2]) < 1e-4):
    output.write("\nMolecule is spherical top.\n")
elif (abs(eigvals[0]-eigvals[1]) < 1e-4 and abs(eigvals[1]-eigvals[2]) > 1e-4):
    output.write("\nMolecule is an oblate symmetric top.\n")
elif (abs(eigvals[0]-eigvals[1]) > 1e-4 and abs(eigvals[1]-eigvals[2]) < 1e-4):
    output.write("\nMolecule is a prolate symmetric top.\n")
else:
    output.write("\nMolecule is an asymmetric top.\n")


In [15]:
f.close()
output.close()